# 3. VAT-MCD Dataset

In [1]:
import os
import pandas as pd
import numpy as np
import scipy.io

## 3.1 개별 데이터 확인
- VAT-MCD 는 매틀랩 데이터형식(.mat)으로되어 있어 해당 파일을 파싱하는 작업이 필요하다.

In [179]:
sampling_rate = 25.6 * 1000 # 25.6 kHz

vib_sample_path = '/home/lilmae/Desktop/Rotray_Machine_Data/dataset/VAT-MCD/vibration/0Nm_BPFI_03.mat'
vib_data = scipy.io.loadmat(vib_sample_path)

## 3.1 개별 데이터 확인
- 먼저 개별 데이터(파일)이 어떤 형태로 존재하는 지 확인해 본다.
- 확인 후, 데이터를 센서 단위로 잘라서 np.float32로 바꾸는 작업까지 해둔다.
> 개별 파일에서 x_A, y_A, x_B, y_B 를 추출하는 작업을 함수화   
> 각 센서의 위치와 방향은 아래 사진과 같다.

![Sample Image](figs/3_sensor.png )

In [177]:
print(f'vib_data.keys() {vib_data.keys()}')
signal_data = vib_data['Signal']

sensor_data = signal_data[0][0][1][0][0][0]
sensor_data = sensor_data.transpose()

x_A_np = sensor_data[0].astype('float32')
y_A_np = sensor_data[1].astype('float32')
x_B_np = sensor_data[2].astype('float32')
y_B_np = sensor_data[3].astype('float32')

vib_data.keys() dict_keys(['__header__', '__version__', '__globals__', 'Signal'])


## 3.2 데이터 interpolation
- 서로 다른 sampling rate 를 가지는 데이터를 활용하기 때문에 이 sampling rate 를 맞추어 주는 것이 필요하다.
- target sampling rate 를 기준으로 데이터를 interpolation 할 수 있도록 한다.

In [183]:
from scipy.interpolate import interp1d

signal = x_A_np # 센서 하나씩 interpolation할 수 있도록 한다 : 예시 x-sensor
time_np = np.arange(len(signal))/sampling_rate
target_sampling = 10*1000 # 기준으로 정한 sampling rate : 예시 10kHz
target_time = np.linspace(0, time_np[-1], target_sampling, endpoint=False) # target sampling rate에 맞도록 시간축 생성

interpolator = interp1d(time_np, signal, kind='linear')
interpolated_signal = interpolator(target_time)

## 3.3 전체 데이터를 순회해서 접근
- 전체 데이터를 순회하여 접근하고 데이터 파일과 클래스 정보를 확인하는 작업을 해준다.
    > 클래스가 unbalance 인 경우 6g 과 27g 의 하부 클래스가 존재함
- 이를 파일 이름, 파일 경로, 클래스 정보 로 데이터베이스화 한다.

In [213]:
import os

vat_root = os.path.join(os.getcwd(), 'dataset', 'VAT-MCD', 'vibration')

data ={
    'file_name' : [],
    'file_path' : [],
    'class' : [],
    'severity' : [],
    'load' : [],
    'sampling_rate' : sampling_rate,
    'speed' : '680-2460'
}


for file_name in os.listdir(vat_root):
    file_path = os.path.join(vat_root, file_name)
    
    data['file_name'].append(file_name)
    data['file_path'].append(file_path)
    
    file_info = file_name.split('_')
    severity = None
    if len(file_info) ==2:
        class_name = 'normal'
        load = file_info[0]
    elif len(file_info) ==3:
        load = file_info[0] 
        class_name = file_info[1]
        severity = file_info[2]
    else:
        print('err : {file_name}')
        
        
    data['class'].append(class_name)
    data['severity'].append(severity)
    data['load'].append(load)
    

data_pd = pd.DataFrame(data)

In [212]:
data_pd.head()

,file_name,file_path,class,severity,load,sampling_rate,speed
0,2Nm_Misalign_03.mat,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,Misalign,03.mat,2Nm,25600.0,680-2460
1,4Nm_Normal.mat,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,normal,None,4Nm,25600.0,680-2460
2,0Nm_Misalign_03.mat,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,Misalign,03.mat,0Nm,25600.0,680-2460
3,0Nm_BPFI_10.mat,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,BPFI,10.mat,0Nm,25600.0,680-2460
4,4Nm_Misalign_03.mat,/home/lilmae/Desktop/Rotray_Machine_Data/datas...,Misalign,03.mat,4Nm,25600.0,680-2460


## 3.4. 전체 함수화
1. `generate_vat_df` : 전체 데이터를 순회해 pd.DataFrame 으로 만드는 함수
2. `open_vat_file` : 개별 데이터를 열어 time,x,y,z 의 np.array 로 추출해 주는 함수
3. `interpol_vat` :센서 데이터를 지정하는 sampling rate로 interpolation 해주는 함수

In [214]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# 1. generate_vat_df : 전체 데이터를 순회해 pd.DataFrame 으로 만드는 함수
def generate_vat_df(vat_root):

    data ={
        'file_name' : [],
        'file_path' : [],
        'class' : [],
        'severity' : [],
        'load' : [],
        'sampling_rate' : sampling_rate,
        'speed' : '680-2460'
    }


    for file_name in os.listdir(vat_root):
        file_path = os.path.join(vat_root, file_name)
        
        data['file_name'].append(file_name)
        data['file_path'].append(file_path)
        
        file_info = file_name.split('_')
        severity = None
        if len(file_info) ==2:
            class_name = 'normal'
            load = file_info[0]
        elif len(file_info) ==3:
            load = file_info[0] 
            class_name = file_info[1]
            severity = file_info[2]
        else:
            print('err : {file_name}')
            
            
        data['class'].append(class_name)
        data['severity'].append(severity)
        data['load'].append(load)
        

    data_pd = pd.DataFrame(data)
    
    return data_pd

# 2. open_vat_file : 개별 데이터를 열어 x_A_np, y_A_np, x_B_np, y_B_np 의 np.array 로 추출해 주는 함수
def open_vat_file(file_path):
    
    vib_data = scipy.io.loadmat(file_path)
    signal_data = vib_data['Signal']

    sensor_data = signal_data[0][0][1][0][0][0]
    sensor_data = sensor_data.transpose()

    x_A_np = sensor_data[0].astype('float32')
    y_A_np = sensor_data[1].astype('float32')
    x_B_np = sensor_data[2].astype('float32')
    y_B_np = sensor_data[3].astype('float32')
    
    return x_A_np, y_A_np, x_B_np, y_B_np

# 3. interpol_vbl :센서 데이터를 지정하는 sampling rate로 interpolation 해주는 함수
def interpol_vat(signal_np, target_sampling):

    time_np = np.arange(len(signal_np))/sampling_rate
    target_sampling = 10*1000 # 기준으로 정한 sampling rate : 예시 10kHz
    target_time = np.linspace(0, time_np[-1], target_sampling, endpoint=False) # target sampling rate에 맞도록 시간축 생성

    interpolator = interp1d(time_np, signal_np, kind='linear')
    interpolated_signal = interpolator(target_time)
    
    return interpolated_signal, target_time